# LexAI - Automated Contract Analysis & Compliance Assistant

LexAI is an advanced AI-powered legal assistant designed to automate contract analysis and streamline compliance evaluation. By leveraging Google's Gemini 2.0 Flash model, Retrieval-Augmented Generation (RAG), and a robust document processing pipeline, LexAI intelligently extracts, analyzes, and compares contract clauses with similar documents to identify potential compliance risks. The system assigns a risk score on a 1–10 scale with clear justifications, enabling standardized evaluations across legal teams. LexAI features a natural language interface, automated document ingestion (PDF/TXT), clause classification, and similarity search using ChromaDB, with metadata stored in SQLite. Its LangGraph-driven workflow coordinates multi-step analysis, ensuring accurate and consistent results. Legal professionals benefit from risk-highlighted contract comparisons, detailed audit trails, and summary reports—ultimately reducing contract review time by up to 80% while improving consistency and uncovering non-obvious legal risks. The system is adaptable across contract types and jurisdictions, maintaining full document integrity through read-only processing, and is currently in advanced stages of implementation and refinement.

In [35]:

import pandas as pd
import numpy as np

I utilized CUAD dataset (Contract Understanding Atticus Dataset) for this task:
Hendrycks, D., Burns, C., Basart, S., Zou, A., Mazeika, M., Song, D., & Steinhardt, J. (2021). CUAD: An Expert-Annotated NLP Dataset for Legal Contract Review. arXiv preprint arXiv:2103.06268. [https://arxiv.org/abs/2103.06268](http://)

* This project uses a processed version of the CUAD dataset, which originally includes:

* 28 labeled Excel files containing clause-level annotations across legal categories

* 510 full contract PDFs and TXTs used as raw source data

* Clauses and labels were vertically merged into a unified format, where the first column represents the Filename and subsequent columns include categorized clause text and annotations

In [36]:

clauses_df = pd.read_excel('/kaggle/input/cuad-data/CUAD_v1/merged_files.xlsx')

In [37]:

clauses_df.head()

,Filename,Change of Control,Anti-assignment,Audit Rights,Covenant not to Sue,Agreement Date,Agreement Date-Answer,Effective Date,Effective Date-Answer,Expiration Date,...,Revenue-Profit Sharing,ROFR-ROFO-ROFN,Source Code Escrow,Termination for Convenience,Third Party Beneficiary,Uncapped Liability,Cap on Liability,Unlimited/All-You-Can-Eat License,Volume Restriction,Warranty Duration
0,2ThemartComInc_19990826_10-12G_EX-10.10_670028...,If a majority of the equity securities of eith...,All rights (under any applicable intellectual ...,"Once every twelve (12) months, 2TheMart throug...",NaN,"June 21, 1999 (Page 1)",1999-06-21 00:00:00,"June 21, 1999 (Page 1)",1999-06-21 00:00:00,The term of this Agreement shall continue for ...,...,"After the Launch Date, i-Escrow shall pay 2The...",NaN,NaN,NaN,NaN,"EXCEPT IN THE EVENT OF A BREACH OF SECTION 11,...","EXCEPT IN THE EVENT OF A BREACH OF SECTION 11,...",NaN,NaN,NaN
1,ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEMENT,NaN,"Provider may not assign, delegate or otherwise...",NaN,NaN,"October 1, 2019 (Page 1)",2019-10-01 00:00:00,"November 1, 2019 (Page 1)",2019-11-01 00:00:00,This Agreement be deemed effective as of the E...,...,NaN,NaN,NaN,"Each of the Recipient and the Provider may, in...",NaN,NaN,NaN,NaN,NaN,NaN
2,ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-...,NaN,No Joint Venturer shall be authorized or ...,"If requested by a Joint Venturer, the Joi...",NaN,NaN,NaN,The Joint Venture shall commence on the 1st of...,2003-03-01 00:00:00,The Joint Venture shall commence on the 1st of...,...,Division of Income and Losses. All income and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACCURAYINC_09_01_2010-EX-10.31-DISTRIBUTOR AGR...,NaN,"Neither this Agreement, nor any of the rights,...",NaN,NaN,"June 8, 2010 (Page 23)",2010-06-08 00:00:00,"June 8, 2010 (Page 1)",2010-06-08 00:00:00,Unless otherwise agreed in writing by Accuray ...,...,NaN,NaN,NaN,NaN,NaN,WITHOUT AFFECTING STRICT PRODUCT LIABILITY UND...,If a Customer notifies Accuray in writing duri...,NaN,NaN,Accuray will provide a warranty to each Custom...
4,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,NaN,Neither ADAMS GOLF nor CONSULTANT shall have t...,NaN,NaN,"January 13, 2005 (Page 2)",2005-01-13 00:00:00,The Term of this Agreement shall be for a peri...,2004-09-01 00:00:00,The Term of this Agreement shall be for a peri...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"During the term of this Agreement, CONSULTANT...",NaN


In [38]:

clauses_df.columns

Index(['Filename', 'Change of Control', 'Anti-assignment', 'Audit Rights',
       'Covenant not to Sue', 'Agreement Date', 'Agreement Date-Answer',
       'Effective Date', 'Effective Date-Answer', 'Expiration Date',
       'Expiration Date-Answer', 'Renewal Term', 'Renewal Term-Answer',
       'Notice Period to Terminate Renewal',
       'Notice Period to Terminate Renewal- Answer', 'Document Name',
       'Governing Law', 'Insurance', 'IP Ownership Assignment',
       'Joint IP Ownership', 'License Grant', 'Non-Transferable License',
       'Affiliate License-Licensor', 'Affiliate License-Licensee',
       'Irrevocable or Perpetual License', 'Liquidated Damages',
       'Minimum Commitment', 'Most Favored Nation', 'No-Solicit of Employees',
       'Competitive Restriction Exception', 'Non-Compete', 'Exclusivity',
       'No-Solicit of Customers', 'Non-Disparagement', 'Parties',
       'Parties-Answer', 'Post-termination Services', 'Price Restrictions',
       'Revenue-Profit Sharing'

In [39]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0" "chromadb==0.6.3" 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

In [40]:
from google import genai
from google.genai import types
from IPython.display import Markdown
genai.__version__

'1.7.0'

In [41]:

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [104]:

client = genai.Client(api_key=GOOGLE_API_KEY)


## SQL connection setup

In [43]:

%reload_ext sql    
%sql sqlite:///contract_clauses.db

In [44]:

import sqlite3

# 1. Load your XLSX file into a pandas DataFrame
xlsx_file = '/kaggle/input/cuad-data/CUAD_v1/merged_files.xlsx'  # Replace with your file path
df = pd.read_excel(xlsx_file)

# 2. Connect to your SQLite database
conn = sqlite3.connect('contract_clauses.db')

# 3. Write the DataFrame to SQL
table_name = 'contract_clauses'  # Choose a name for your table
df.to_sql(table_name, conn, if_exists='replace', index=False)

# 4. Verify the data was loaded
query = f"SELECT * FROM {table_name} LIMIT 5;"
print(pd.read_sql(query, conn))

# 5. Close the connection
conn.close()

                                            Filename  \
0  2ThemartComInc_19990826_10-12G_EX-10.10_670028...   
1   ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEMENT   
2  ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-...   
3  ACCURAYINC_09_01_2010-EX-10.31-DISTRIBUTOR AGR...   
4  ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...   

                                   Change of Control  \
0  If a majority of the equity securities of eith...   
1                                               None   
2                                               None   
3                                               None   
4                                               None   

                                     Anti-assignment  \
0  All rights (under any applicable intellectual ...   
1  Provider may not assign, delegate or otherwise...   
2  No Joint      Venturer shall be authorized or ...   
3  Neither this Agreement, nor any of the rights,...   
4  Neither ADAMS GOLF nor CONSULTANT shall hav

In [45]:

db_conn = sqlite3.connect('contract_clauses.db')

In [46]:

# def describe_table(table_name: str) -> list[tuple[str, str]]:
#     """Look up the table schema.

#     Returns:
#       List of columns, where each entry is a tuple of (column, type).
#     """
#     print(f' - DB CALL: describe_table({table_name})')

#     cursor = db_conn.cursor()

#     cursor.execute(f"PRAGMA table_info({table_name});")

#     schema = cursor.fetchall()
#     # [column index, column name, column type, ...]
#     return [(col[1], col[2]) for col in schema]

# describe_table('contract_clauses')

In [47]:

# def execute_query(sql: str) -> list[list[str]]:
#     """Execute an SQL statement, returning the results."""
#     print(f' - DB CALL: execute_query({sql})')

#     cursor = db_conn.cursor()

#     cursor.execute(sql)
#     return cursor.fetchall()

# execute_query('select * from contract_clauses limit 1')

In [48]:

# def list_tables() -> list[str]:
#     """Retrieve the names of all tables in the database."""
#     # Include print logging statements so you can see when functions are being called.
#     print(' - DB CALL: list_tables()')

#     cursor = db_conn.cursor()

#     # Fetch the table names.
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

#     tables = cursor.fetchall()
#     return [t[0] for t in tables]


# list_tables()

In [49]:

db_tools= [list_tables, describe_table, execute_query]

## Retrieval Augmented Generation

In [50]:

import os

def get_txt_documents(directory_path):
    """
    Extract text from all TXT files in a directory and prepare for ChromaDB
    Preserves original filenames (without .txt) as document IDs
    
    Args:
        directory_path (str): Path to directory containing TXT files
        
    Returns:
        tuple: (documents, metadatas, ids) in ChromaDB format
    """
    documents = []
    metadatas = []
    ids = []
    
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            
            try:
                # Read text directly from TXT file
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                
                # Prepare ChromaDB entries
                documents.append(text)
                metadatas.append({
                    "source": file_path,
                    "filename": filename
                })
                # Remove ".txt" from the ID
                id_without_extension = os.path.splitext(filename)[0]
                ids.append(id_without_extension)
                
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    return documents, metadatas, ids

# Usage
txt_directory = "/kaggle/input/cuad-data/CUAD_v1/full_contract_txt"
documents, metadatas, ids = get_txt_documents(txt_directory)


In [51]:

from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [52]:

import math

import chromadb

DB_NAME = "contracts"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)


batch_size = 30  # Adjust based on document size and API limits (100 is often doc limit)
total_docs = len(documents)

print(f"Adding {total_docs} documents...")

for i in range(0, total_docs, batch_size):
    # Get the current batch slice
    doc_batch = documents[i : i + batch_size]
    meta_batch = metadatas[i : i + batch_size]
    id_batch = ids[i : i + batch_size]

    print(f"  Adding batch {i // batch_size + 1}/{math.ceil(total_docs / batch_size)}")

    # Add the batch to ChromaDB
    db.add(
        documents=doc_batch,
        metadatas=meta_batch,
        ids=id_batch
    )

# Verify
print(f"Loaded {len(documents)} TXT files into ChromaDB")

Adding 510 documents...
  Adding batch 1/17
  Adding batch 2/17
  Adding batch 3/17
  Adding batch 4/17
  Adding batch 5/17
  Adding batch 6/17
  Adding batch 7/17
  Adding batch 8/17
  Adding batch 9/17
  Adding batch 10/17
  Adding batch 11/17
  Adding batch 12/17
  Adding batch 13/17
  Adding batch 14/17
  Adding batch 15/17
  Adding batch 16/17
  Adding batch 17/17
Loaded 510 TXT files into ChromaDB


In [53]:
db.count()

510

In [54]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "what is the reseller agreement with detto technologies?"

result = db.query(query_texts=[query], n_results=2)
[all_passages] = result["documents"]

Markdown(all_passages[0][:500])

Exhibit 10d-2

                               RESELLER AGREEMENT

                                 BY AND BETWEEN

                                PIVX CORPORATION

                                       AND

                               DETTO TECHNOLOGIES

This Reseller Agreement is entered as of this ___ day of _________, 2004 ("Effective Date") by and between PivX Corporation, a California corporation, having its principal place of business at 23 Corporate Plaza Drive, Newport Beach, Califo

## Implementing Langraph

In [117]:

from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class ContractState(TypedDict):
    """State representation of a contract review conversation"""

    messages: Annotated[list, add_messages]
    document: tuple[str, str] | None  
    documents: list[tuple[str, str, str]]
    finished:bool

LEXAIBOT_SYSINT = (
'system',
"""You are a contract review assistant. Your role is to help users understand and analyze legal contracts by leveraging both document content and structured clause data.
You review contracts, provide risk scoring, and highlight key terms and conditions.

You have access to a ChromaDB collection containing full contract documents and a SQL database with structured clause data.
The ChromaDB collection contains the full text of contracts. The SQL database contains structured clause data with various columns representing different clauses and sections of contracts.
The use may upload a document (optional), and you will extract the text from it. It will always ask a question.
You will prepare a minimum 2 lines and maximum 20 lines query, viewing the document uploaded (if any) and the question asked. This query wull be used to retrieve relevent documents using retrieval() function.
When a user asks a question:

1. Use the retrieval() function to query the ChromaDB collection 'contracts' to find the most relevant contract documents using semantic search
   - The retrieval function will return top k documents along with their IDs as "Filename", metadata

2. Use the SQL database functions to get structured clause data:
   - First call list_tables() to identify available tables
   - Use describe_table() to verify the "Filename" column exists
   - Use execute_query() to select all clauses (full rows) from the table where Filename matches the retrieved document IDs
   
3. Analyze both:
   - The full contract text from ChromaDB results 
   - The categorized clause data from the SQL database rows

Available tools:
- retrieval() function to search ChromaDB collection, and retrieve top 'k' asccoiated documents
- SQL database functions:
  - execute_query() - Run SQL queries to select clause data
  - describe_table() - Verify table schema and columns
  - list_tables() - Get available tables

When responding:
- Cite specific clauses and sections
- Explain legal concepts in plain language
- Highlight key terms and conditions
- Note any important caveats or limitations
- If you cannot find relevant information, say so clearly
- Make legal interpretations or provide legal advice
- Provide risk scoring based on scale of 1-10, where 1 is low risk and 10 is high risk

Do not:

- Speculate beyond what is in the documents
- Share sensitive contract details inappropriately

Always maintain a professional, factual tone focused on helping users understand contract content based on the available documentation."""
)
WELCOME_MSG = """Hello! I'm Lexaibot, your legal assistant. How can I help you with your legal contract review today?"""

from langgraph.graph import StateGraph, START, END
from google.api_core import retry
from google.api_core.exceptions import ServiceUnavailable
from langchain_google_genai import ChatGoogleGenerativeAI





llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)


from  typing import Literal
from  langchain_core.messages.ai import AIMessage
from collections.abc import Iterable
from random import randint
from langgraph.prebuilt import ToolNode
from langchain_core.messages.tool import ToolMessage
from langchain_core.tools import tool

@tool
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""

@tool
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Retrieve the schema and column names of a table.
    Args:
        table_name (str): Name of the table to describe.
    Returns:
        List of tuples containing column names and types.
    """

@tool
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results.
    Args:
        sql (str): SQL statement to execute.
    Returns:
        List of rows, where each row is a list of column values.
    """

# @tool
# def retrieval(query: str, k: int = 3) -> list[tuple[str, str, str]]:
#     """Retrieve top k relevant documents from ChromaDB based on query.
#     Args:
#         query (str): The query string to search for.
#         k (int): The number of top results to return.
#     Returns:
#         list[tuple[str, str, str]]: List of tuples containing document id, text and metadata.
#     """




tool_node = ToolNode([list_tables, describe_table, execute_query])


llm_with_tools = llm.bind_tools(contract_tools)

def db_tools_node(state: ContractState) -> ContractState:
    """"The db tools node. This is a wrapper around the SQL database tools: list_tables, describe_table, execute_query."""
    # print("Came to db tools node")
    tool_msg = state.get("messages", [])[-1]
    outbound_msgs = []
    # code to appropiately call functions list_tables, describe_table, execute_query.
    if not isinstance(tool_msg, ToolMessage):
        raise ValueError(f"Expected a ToolMessage, got {type(tool_msg)}")
    if not tool_msg.tool_calls:
        raise ValueError(f"ToolMessage has no tool calls: {tool_msg}")
    for tool_call in tool_msg.tool_calls:
        tool_name = tool_call["name"]
        if tool_name == "list_tables":
            # Call the list_tables function
            # print("Calling List Tables function")
            cursor = db_conn.cursor()
            # Fetch the table names.
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()
            result = [t[0] for t in tables]
            outbound_msgs.append(ToolMessage(content=result))
        elif tool_name == "describe_table":
            # print("Calling Describe_table function")
            # Call the describe_table function
            table_name = tool_call["args"][0]
            cursor = db_conn.cursor()

            cursor.execute(f"PRAGMA table_info({table_name});")

            schema = cursor.fetchall()
            # [column index, column name, column type, ...]
            result = [(col[1], col[2]) for col in schema]
            outbound_msgs.append(ToolMessage(content=result))
        elif tool_name == "execute_query":
            # print("Calling execute_query function")
            # Call the execute_query function
            sql = tool_call["args"][0]
            cursor = db_conn.cursor()
            cursor.execute(sql)
            outbound_msgs.append(ToolMessage(content=cursor.fetchall()))
        else:
            raise ValueError(f"Unknown tool call: {tool_call}")
    return state | {"messages": outbound_msgs}

def retrieval_node(state: ContractState) -> ContractState:
    """The retrieval node. This is a wrapper around the retrieval function."""
    # print("Retrieving documents")
    tool_msg = state.get("messages", [])[-1]
    outbound_msgs = []
    
    if not isinstance(tool_msg, ToolMessage):
        raise ValueError(f"Expected a ToolMessage, got {type(tool_msg)}")
    if not tool_msg.tool_calls:
        raise ValueError(f"ToolMessage has no tool calls: {tool_msg}")
        
    for tool_call in tool_msg.tool_calls:
        tool_name = tool_call["name"]
        if tool_name == "retrieval":
            # Call the retrieval function
            query = tool_call["args"][0]
            k = tool_call["args"][1]
            # print('Retreiving docs...')
            result = db.query(query_texts=[query], n_results=k)
            [all_passages] = result["documents"]
            
            # Extract metadata and ids
            metadata = result["metadatas"]
            ids = result["ids"]
            
            # Combine text, metadata and ids into tuples
            combined_results = [(doc_id, passage, meta) for doc_id, passage, meta in zip(ids, all_passages, metadata)]
            
            # Add the results to both state and outbound messages
            # state['documents'] = combined_results
            outbound_msgs.append(ToolMessage(
                tool_call_id=tool_call["id"],
                content=str(combined_results)  # Convert results to string for message
            ))
        else:
            raise ValueError(f"Unknown tool call: {tool_call}")
    
    # Return state with both documents and messages updated
    return {'messages' : outbound_msgs}


def human_node(state: ContractState) -> ContractState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    # print("\n=== Current Conversation State ===")
    print("Model:", last_msg.content)

    user_input = input("\nUser: ")
    # print(f"Received input: {user_input}")  # Debug print

    # Only set finished if user explicitly wants to quit
    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
        print("User requested to end conversation")
        state["finished"] = True
        return state | {"messages": [("user", user_input)]}
    
    # Continue conversation
    # print("Continuing conversation...")  # Debug print
    return state | {"messages": [("user", user_input)]}


def chatbot(state: ContractState) -> ContractState:
    """The chatbot with tools. A simple wrapper around the model's own chat interface."""
    defaults = {"documents": [], "finished": False, 'document' : None}
    # print(state)
    if state["document"] is not None:
        # print('uploading file')
        document_file = client.files.upload(file=state["document"][0])
        state['document'][1] = document_file
    
    if state["messages"] and (state['document'] is not None):
        # print('Augmenting document')
        new_output = llm_with_tools.invoke([LEXAIBOT_SYSINT] + state["messages"]) + "Document: "+ state["document"][1]
    elif state["messages"]:
        # print('Just messaging')
        new_output = llm_with_tools.invoke([('system','lets play a game')] + state["messages"])
        # print('new_output')
    else:
        new_output = AIMessage(content=WELCOME_MSG)
    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.

    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.
    return defaults | state | {"messages": [new_output]}



def maybe_exit_human_node(state: ContractState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"
    
    

def maybe_route_to_nodes(state: ContractState) -> Literal["db_tools", "human", "retrieval_node"]:
    """Route between chat and tool nodes if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]
    # print('too_calls:', msg.tool_calls)
    if state.get("finished", False):
        return END
    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        # Route to appropriate tool node based on tool name
        
        tool_name = msg.tool_calls[0]["name"]
        if tool_name in ["list_tables", "describe_table", "execute_query"]:
            return "db_tools"
        elif tool_name == "retrieval":
            return "retrieval_node"
        else:
            return "human"

    else:
        return "human"



In [118]:
# Define the state graph with the nodes and edges

graph_builder = StateGraph(ContractState)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("human", human_node)
graph_builder.add_node("db_tools", db_tools_node)
graph_builder.add_node("retrieval_node", retrieval_node)

# Chatbot -> {ordering, tools, human, END}
graph_builder.add_conditional_edges("chatbot", maybe_route_to_nodes)
# Human -> {chatbot, END}
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Tools (both kinds) always route back to chat afterwards.
graph_builder.add_edge("db_tools", "chatbot")
graph_builder.add_edge("retrieval_node", "chatbot")

graph_builder.add_edge(START, "chatbot")
graph_with_contract_tools = graph_builder.compile()

# from  IPython.display import Image, display
# Image(graph_with_contract_tools.get_graph().draw_mermaid_png())

In [ ]:
# Create initial state
initial_state = ContractState(
    messages=[],  # empty initial messages
    documents=[],
    document = None,
    finished=False
)

conv = graph_with_contract_tools.invoke(initial_state)

Model: Hello! I'm Lexaibot, your legal assistant. How can I help you with your legal contract review today?
